In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
pip install -i https://test.pypi.org/simple/ my_krml_25076922==2024.0.3.3

Looking in indexes: https://test.pypi.org/simple/
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np

import os

/Users/vega7unk/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
path ='/Users/vega7unk/Documents/3rd_Sem_DSI/Adv_ML/AT2/Project/adv_mla_at2/data'

sales_data = pd.read_csv(f'{path}/df_cleaned.csv')

In [5]:
from my_krml_25076922.features.data_processing import scale_features

# List of features to scale
features_to_scale = ['total_sales', 'day_of_week', 'month','year']

# Apply scaling to the numerical columns
sales_data = scale_features(sales_data, features_to_scale)

# Preview the scaled data
sales_data.head()

,item_id,dept_id,cat_id,store_id,state_id,total_sales,day_of_week,month,year
0,1437,3,1,0,0,-0.330511,0.993873,-1.539218,-1.371861
1,1438,3,1,0,0,-0.330511,0.993873,-1.539218,-1.371861
2,1439,3,1,0,0,-0.330511,0.993873,-1.539218,-1.371861
3,1440,3,1,0,0,-0.330511,0.993873,-1.539218,-1.371861
4,1441,3,1,0,0,-0.330511,0.993873,-1.539218,-1.371861


In [6]:
target = sales_data.pop('total_sales')

In [7]:
from my_krml_25076922.data.sets import split_sets_random

X_train, y_train, X_val, y_val, X_test, y_test = split_sets_random(sales_data, target, test_ratio=0.2)

In [8]:
import xgboost as xgb

xgb_model = xgb.XGBRegressor(
    n_estimators=100,          # Number of trees
    learning_rate=0.1,         # Step size shrinkage
    max_depth=6,               # Maximum tree depth for base learners
    subsample=0.8,             # Subsample ratio of the training instance
    colsample_bytree=0.8,      # Subsample ratio of columns when constructing each tree
    random_state=42
)

# Fit the model on the training data
xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=10, verbose=True)

# Make predictions on the validation set
y_pred = xgb_model.predict(X_val)

/Users/vega7unk/anaconda3/lib/python3.11/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-rmse:0.99556
[1]	validation_0-rmse:0.99337
[2]	validation_0-rmse:0.99121
[3]	validation_0-rmse:0.98948
[4]	validation_0-rmse:0.98802
[5]	validation_0-rmse:0.98697
[6]	validation_0-rmse:0.98544
[7]	validation_0-rmse:0.98421
[8]	validation_0-rmse:0.98317
[9]	validation_0-rmse:0.98244
[10]	validation_0-rmse:0.98193
[11]	validation_0-rmse:0.98150
[12]	validation_0-rmse:0.98108
[13]	validation_0-rmse:0.98019
[14]	validation_0-rmse:0.97956
[15]	validation_0-rmse:0.97907
[16]	validation_0-rmse:0.97869
[17]	validation_0-rmse:0.97817
[18]	validation_0-rmse:0.97761
[19]	validation_0-rmse:0.97730
[20]	validation_0-rmse:0.97677
[21]	validation_0-rmse:0.97638
[22]	validation_0-rmse:0.97620
[23]	validation_0-rmse:0.97602
[24]	validation_0-rmse:0.97565
[25]	validation_0-rmse:0.97546
[26]	validation_0-rmse:0.97496
[27]	validation_0-rmse:0.97484
[28]	validation_0-rmse:0.97474
[29]	validation_0-rmse:0.97461
[30]	validation_0-rmse:0.97443
[31]	validation_0-rmse:0.97424
[32]	validation_0-

In [9]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Evaluate the model
rmse = mean_squared_error(y_val, y_pred, squared=False)  # RMSE
mae = mean_absolute_error(y_val, y_pred)

print(f"RMSE Validation: {rmse}")
print(f"MAE Validation: {mae}")

RMSE Validation: 0.9674759756375952
MAE Validation: 0.44916161068867544


/Users/vega7unk/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [12]:
import joblib

joblib.dump(xgb_model, '/Users/vega7unk/Documents/3rd_Sem_DSI/Adv_ML/AT2/Project/adv_mla_at2/models/predictive/xgb_predictive_model.pkl')

['/Users/vega7unk/Documents/3rd_Sem_DSI/Adv_ML/AT2/Project/adv_mla_at2/models/predictive/xgb_predictive_model.pkl']